Uploads Style and Content files to google drive as style.wav and content.wav

In [ ]:
pip install audiotsm

In [ ]:
!pip install ffmpeg

In [ ]:
pip install spleeter

In [ ]:
from google.colab import files
#from IPython.display import Audio #for testing
import os
import shutil
# Imports
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import cv2

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16

from scipy.optimize import fmin_l_bfgs_b
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import librosa
from librosa import display
import matplotlib.pyplot as plt
%matplotlib inline

import math
import pywt
from scipy import signal

from audiotsm import phasevocoder
from audiotsm.io.wav import WavReader, WavWriter

In [ ]:
from IPython.display import Audio

In [ ]:
import torch
import copy

In [ ]:
import torch.nn as nn
from torch.nn import Conv2d, ReLU, AvgPool1d, MaxPool2d, Linear, Conv1d
from torch.autograd import Variable
import torch.optim as optim
import torchvision.transforms as transforms

# Upload

In [ ]:
#upload style file and rename to style.wav
a = 1
while a:
  style = files.upload()
  for fn in style.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(style[fn])))
  if len(style.keys()) > 1:
    print('UPLOAD ONLY ONE FILE')
  else:
    stylename = fn
    a = 0
  del style

In [ ]:
#upload content file and rename to content.wav
b = 1
while b:
  content = files.upload()
  for fnb in content.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fnb, length=len(content[fnb])))
  if len(content.keys()) > 1:
    print('UPLOAD ONLY ONE FILE')
  else:
    contentname = fnb
    b = 0
  del content

In [ ]:
try:
    os.remove("style.wav")
    os.remove("content.wav")
except OSError:
    pass

shutil.move(stylename, "style.wav")
shutil.move(contentname, "content.wav")


you can add an 'Audio(filemane)' at any point to listen to valid audio files:


In [ ]:
#Audio('style.wav')

# Sync BPM

In [ ]:
def read_audio_spectum_extended(filename):
    x, fs = librosa.load(filename)
    S = librosa.stft(x, N_FFT)
    S = np.log1p(np.abs(S)) 
    return S, x, fs

In [ ]:
# SOURCE: https://github.com/scaperot/the-BPM-detector-python/blob/master/bpm_detection/bpm_detection.py
def no_audio_data():
    print("No audio data for sample, skipping...")
    return None, None

def peak_detect(data):
    max_val = np.amax(abs(data))
    peak_ndx = np.where(data == max_val)
    if len(peak_ndx[0]) == 0:  # if nothing found then the max must be negative
        peak_ndx = np.where(data == -max_val)
    return peak_ndx

def bpm_detector(data, fs):
    cA = []
    cD = []
    correl = []
    cD_sum = []
    levels = 4
    max_decimation = 2 ** (levels - 1)
    min_ndx = math.floor(60.0 / 220 * (fs / max_decimation))
    max_ndx = math.floor(60.0 / 40 * (fs / max_decimation))

    for loop in range(0, levels):
        cD = []
        # 1) DWT
        if loop == 0:
            [cA, cD] = pywt.dwt(data, "db4")
            cD_minlen = len(cD) / max_decimation + 1
            cD_sum = np.zeros(math.floor(cD_minlen))
        else:
            [cA, cD] = pywt.dwt(cA, "db4")

        # 2) Filter
        cD = signal.lfilter([0.01], [1 - 0.99], cD)

        # 4) Subtract out the mean.

        # 5) Decimate for reconstruction later.
        cD = abs(cD[:: (2 ** (levels - loop - 1))])
        cD = cD - np.mean(cD)

        # 6) Recombine the signal before ACF
        #    Essentially, each level the detail coefs (i.e. the HPF values) are concatenated to the beginning of the array
        cD_sum = cD[0 : math.floor(cD_minlen)] + cD_sum

    if [b for b in cA if b != 0.0] == []:
        return no_audio_data()

    # Adding in the approximate data as well...
    cA = signal.lfilter([0.01], [1 - 0.99], cA)
    cA = abs(cA)
    cA = cA - np.mean(cA)
    cD_sum = cA[0 : math.floor(cD_minlen)] + cD_sum

    # ACF
    correl = np.correlate(cD_sum, cD_sum, "full")

    midpoint = math.floor(len(correl) / 2)
    correl_midpoint_tmp = correl[midpoint:]
    peak_ndx = peak_detect(correl_midpoint_tmp[min_ndx:max_ndx])
    if len(peak_ndx) > 1:
        return no_audio_data()

    peak_ndx_adjusted = peak_ndx[0] + min_ndx
    bpm = 60.0 / peak_ndx_adjusted * (fs / max_decimation)
    #print(bpm)
    return bpm, correl

def get_bpm(raw_samples, fs):
    data = []
    correl = []
    bpm = 0
    n = 0
    nsamps = len(raw_samples)
    window_samps = int(window * fs)
    samps_ndx = 0  # First sample in window_ndx
    max_window_ndx = math.floor(nsamps / window_samps)
    bpms = np.zeros(max_window_ndx)
    
    for window_ndx in range(0, max_window_ndx):

        # Get a new set of samples
        # print(n,":",len(bpms),":",max_window_ndx_int,":",fs,":",nsamps,":",samps_ndx)
        data = raw_samples[samps_ndx : samps_ndx + window_samps]
        if not ((len(data) % window_samps) == 0):
            raise AssertionError(str(len(data)))

        bpm, correl_temp = bpm_detector(data, fs)
        if bpm is None:
            continue
        bpms[window_ndx] = bpm
        correl = correl_temp

        # Iterate at the end of the loop
        samps_ndx = samps_ndx + window_samps

        # Counter for debug...
        n = n + 1

    bpm = np.median(bpms)
    
    return bpm

In [ ]:
CONTENT_FILENAME = 'content.wav'
STYLE_FILENAME = 'style.wav'
STYLE_BPM_MOD_FILENAME = 'style_synced.wav'

N_FFT = 2048

window = 3

In [ ]:
a_content, raw_content, fs_content = read_audio_spectum_extended(CONTENT_FILENAME)
a_style, raw_style, fs_style = read_audio_spectum_extended(STYLE_FILENAME)

In [ ]:
bpm_content = get_bpm(raw_content, fs_content);
bpm_style = get_bpm(raw_style, fs_style);
print(bpm_content)
print(bpm_style)

bpm_ratio = bpm_content / bpm_style;
print(bpm_ratio)

In [ ]:
with WavReader(STYLE_FILENAME) as reader:
    with WavWriter(STYLE_BPM_MOD_FILENAME, reader.channels, reader.samplerate) as writer:
        tsm = phasevocoder(reader.channels, speed=bpm_ratio)
        tsm.run(reader, writer)

# Split audio

In [ ]:
!spleeter separate -o /split style_synced.wav

In [ ]:
!spleeter separate -o /split content.wav

# Style transfer via spectrogram

In [ ]:
#the following code and text is largely taken from:
#https://github.com/spiyer99/spiyer99.github.io/blob/master/nbs/Neural%20Transfer%20of%20Audio%20in%20Pytorch.ipynb

In [ ]:
content_audio_name = '/split/content/accompaniment.wav'
style_audio_name = '/split/style_synced/accompaniment.wav'
content_audio_name2 = '/split/content/vocals.wav'
style_audio_name2 = '/split/style_synced/vocals.wav'

## Loss

There are two types of loss for this:

1. Content loss. Lower values for this means that the output audio sounds like joe rogan. 

2. Style loss. Lower values for this means that the output audio sounds like joey diaz.

Ideally we want both content and style loss to be minimised.


In [ ]:
class GramMatrix(nn.Module):

	def forward(self, input):
		a, b, c = input.size()  # a=batch size(=1)
				# b=number of feature maps
				# (c,d)=dimensions of a f. map (N=c*d)
		features = input.view(a * b, c)  # resise F_XL into \hat F_XL
		G = torch.mm(features, features.t())  # compute the gram product
				# we 'normalize' the values of the gram matrix
				# by dividing by the number of element in each feature maps.
		return G.div(a * b * c)
	

# https://ghamrouni.github.io/stn-tuto/advanced/neural_style_tutorial.html#
class ContentLoss(nn.Module):

		def __init__(self, target, weight):
				super(ContentLoss, self).__init__()
				# we 'detach' the target content from the tree used
				self.target = target.detach() * weight
				# to dynamically compute the gradient: this is a stated value,
				# not a variable. Otherwise the forward method of the criterion
				# will throw an error.
				self.weight = weight
				self.criterion = nn.MSELoss()

		def forward(self, input):
				self.loss = self.criterion(input * self.weight, self.target)
				self.output = input
				return self.output

		def backward(self, retain_graph=True):
				self.loss.backward(retain_graph=retain_graph)
				return self.loss


class StyleLoss(nn.Module):

	def __init__(self, target, weight):
		super(StyleLoss, self).__init__()
		self.target = target.detach() * weight
		self.weight = weight
		self.gram = GramMatrix()
		self.criterion = nn.MSELoss()

	def forward(self, input):
		self.output = input.clone()
		self.G = self.gram(input)
		self.G.mul_(self.weight)
		self.loss = self.criterion(self.G, self.target)
		return self.output

	def backward(self,retain_graph=True):
		self.loss.backward(retain_graph=retain_graph)
		return self.loss

## Converting Wav to Matrix

To convert the waveform audio to a matrix that we can pass to pytorch I'll use `librosa`. Most of this code was borrowed from Dmitry Ulyanov's [github repo](https://github.com/DmitryUlyanov/neural-style-audio-tf/blob/master/neural-style-audio-tf.ipynb) and Alish Dipani's [github repo](https://github.com/alishdipani/Neural-Style-Transfer-Audio). 

We get the Short-time Fourier transform from the audio using the `librosa` library. The window size for this is `2048`, which is also the default setting. There is scope here for replacing the code with code from torchaudio. But this works for now.

In [ ]:
import gc; gc.collect()

401

In [ ]:
# USING LIBROSA
N_FFT=2048
def read_audio_spectum(filename):
  x, fs = librosa.load(filename)
  S = librosa.stft(x, N_FFT)
  p = np.angle(S)
  S = np.log1p(np.abs(S))  
  return S, fs

style_audio, style_sr = read_audio_spectum(style_audio_name)
content_audio, content_sr = read_audio_spectum(content_audio_name)

style_audio2, style_sr2 = read_audio_spectum(style_audio_name2)
content_audio2, content_sr2 = read_audio_spectum(content_audio_name2)

if(content_sr != style_sr):
  raise 'Sampling rates are not same'

if(content_sr2 != style_sr2):
  raise 'Sampling rates are not same'

  
style_audio = style_audio.reshape([1,1025,style_audio.shape[1]])
content_audio = content_audio.reshape([1,1025,content_audio.shape[1]])

style_audio2 = style_audio2.reshape([1,1025,style_audio2.shape[1]])
content_audio2 = content_audio2.reshape([1,1025,content_audio2.shape[1]])

if torch.cuda.is_available():
  style_float = Variable((torch.from_numpy(style_audio)).cuda())
  content_float = Variable((torch.from_numpy(content_audio)).cuda())	

  style_float2 = Variable((torch.from_numpy(style_audio2)).cuda())
  content_float2 = Variable((torch.from_numpy(content_audio2)).cuda())
else:
  style_float = Variable(torch.from_numpy(style_audio))
  content_float = Variable(torch.from_numpy(content_audio))

  style_float2 = Variable(torch.from_numpy(style_audio2))
  content_float2 = Variable(torch.from_numpy(content_audio2))

run the following twice, once while using this beforehand:

In [ ]:
style_float = style_float2
content_float = content_float2

## Create CNN

In [ ]:
import gc; gc.collect(); del cnn

This CNN is very shallow. It consists of 2 convolutions and a ReLU in between them. I originally took the CNN used [here](https://github.com/alishdipani/Neural-Style-Transfer-Audio/blob/master/NeuralStyleTransfer.py) but I've made a few changes. 

 - Firstly, I added content loss. This wasn't added before and is obviously very useful. We'd like to know how close (or far away) the audio sounds to the original content.

 - Secondly, I added a ReLU to the model. It's pretty well [established](https://stats.stackexchange.com/questions/275358/why-is-increasing-the-non-linearity-of-neural-networks-desired) that nonlinear activations are desired in a neural network. Adding a ReLU improved the model significantly.

 - Increased the number of steps. From ``2500`` to `20000`

 - Slightly deepened the network. I added a layer of `Conv1d`. After this layer style loss and content loss is calculated

In [ ]:
class CNNModel(nn.Module):
	def __init__(self):
		super(CNNModel, self).__init__()
		self.cnn1 = Conv1d(in_channels=1025, out_channels=4096, kernel_size=3, stride=1, padding=1)
		self.relu = ReLU()
		self.cnn2 = Conv1d(in_channels=4096, out_channels=4096, kernel_size=3, stride=1, padding=1)

	def forward(self, x):
		out = self.cnn1(x)
		out = self.relu(out)
		out = self.cnn2(x)
		out = self.relu(out)
		out = self.cnn3(x)
		return out

In [ ]:
cnn = CNNModel()
if torch.cuda.is_available():
  cnn = cnn.cuda()


style_weight=1000
content_weight = 2


def get_style_model_and_losses(cnn, style_float,\
                               content_float=content_float,\
                               style_weight=style_weight):
  
  cnn = copy.deepcopy(cnn)

  style_losses = []
  content_losses = []

  # create model
  model = nn.Sequential()

  # we need a gram module in order to compute style targets
  gram = GramMatrix()

  # load onto gpu  
  if torch.cuda.is_available():
    model = model.cuda()
    gram = gram.cuda()

  # add conv1
  name = 'conv_1'
  model.add_module(name, cnn.cnn1)

  # add relu
  name = 'relu1'
  model.add_module(name, cnn.relu)

  # add conv2
  name = 'conv_2'
  model.add_module(name, cnn.cnn2)

  # add style loss
  target_feature = model(style_float).clone()
  target_feature_gram = gram(target_feature)
  style_loss = StyleLoss(target_feature_gram, style_weight)
  model.add_module("style_loss_1", style_loss)
  style_losses.append(style_loss)

  # add content loss
  target = model(content_float).detach()
  content_loss = ContentLoss(target, content_weight)
  model.add_module("content_loss_1", content_loss)
  content_losses.append(content_loss)

  return model, style_losses, content_losses


get_style_model_and_losses(cnn, style_float, content_float)

In [ ]:
import gc; gc.collect()

input_float = content_float.clone()
#input_float = Variable(torch.randn(content_float.size())).type(torch.FloatTensor)

learning_rate_initial = 1e-4

def get_input_param_optimizer(input_float):
  input_param = nn.Parameter(input_float.data)
  # optimizer = optim.Adagrad([input_param], lr=learning_rate_initial, lr_decay=0.0001,weight_decay=0)
  optimizer = optim.Adam([input_param], lr=learning_rate_initial)
  # optimizer = optim.LBFGS([input_param], lr=learning_rate_initial)
  # optimizer = optim.SGD([input_param], lr=learning_rate_initial)
  # optimizer = optim.RMSprop([input_param], lr=learning_rate_initial)
  return input_param, optimizer

num_steps= 1000 #number of steps


# from https://pytorch.org/tutorials/advanced/neural_style_tutorial.html
def run_style_transfer(cnn, style_float=style_float,\
                       content_float=content_float,\
                       input_float=input_float,\
                       num_steps=num_steps, style_weight=style_weight): 
  print('Building the style transfer model..')
  # model, style_losses = get_style_model_and_losses(cnn, style_float)
  model, style_losses, content_losses = get_style_model_and_losses(cnn, style_float, content_float)
  input_param, optimizer = get_input_param_optimizer(input_float)
  print('Optimizing..')
  run = [0]

  while run[0] <= num_steps:
    def closure():
            # correct the values of updated input image
      input_param.data.clamp_(0, 1)

      optimizer.zero_grad()
      model(input_param)
      style_score = 0
      content_score = 0

      for sl in style_losses:
        #print('sl is ',sl,' style loss is ',style_score)
        style_score += sl.loss

      for cl in content_losses:
        content_score += cl.loss

      style_score *= style_weight
      content_score *= content_weight

      loss = style_score + content_score
      loss.backward()

      run[0] += 1
      if run[0] % 100 == 0:
        print("run {}:".format(run))
        print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
        print()

      return style_score + content_score

    optimizer.step(closure)

  # ensure values are between 0 and 1
  input_param.data.clamp_(0, 1)

  return input_param.data


In [ ]:
output = run_style_transfer(cnn, style_float=style_float, content_float=content_float, input_float=input_float)

## Reconstruct Audio

Finally the audio needs to be reconstructed. To do that the librosa inverse short-time fourier transform can be used. 

Then we write to an audio file and use the jupyter notebook extension to play the audio in the notebook. 

In [ ]:
# taken from: https://github.com/alishdipani/Neural-Style-Transfer-Audio/blob/master/NeuralStyleTransfer.py

if torch.cuda.is_available():
  output = output.cpu()

output = output.squeeze(0)
output = output.numpy()

N_FFT=2048
a = np.zeros_like(output)
a = np.exp(output) - 1

# This code is supposed to do phase reconstruction
p = 2 * np.pi * np.random.random_sample(a.shape) - np.pi
for i in range(500):
  S = a * np.exp(1j*p)
  x = librosa.istft(S)
  p = np.angle(librosa.stft(x, N_FFT))


In [ ]:
OUTPUT_FILENAME1 = 'test.wav'
OUTPUT_FILENAME2 = 'test2.wav'

change output file name before running a second time

In [ ]:
import soundfile as sf

OUTPUT_FILENAME = OUTPUT_FILENAME1
sf.write(OUTPUT_FILENAME, x, style_sr)

In [ ]:
Audio(OUTPUT_FILENAME1)

#Denoise

# Combine results

In [ ]:
samples1, fs1 = librosa.load(OUTPUT_FILENAME1)
samples2, fs2 = librosa.load(OUTPUT_FILENAME2)

In [ ]:
samples = 0.5 * samples1
samples += 0.5 * samples2

fs = fs1

In [ ]:
# Save output data
import soundfile as sf
sf.write("transfered_sum.wav", samples, fs)

In [ ]:
Audio("transfered_sum.wav")